In [9]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
from amftrack.pipeline.development.high_mag_videos.plot_data import save_raw_data, plot_summary
import os
from pathlib import Path
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
from tifffile import imwrite

%matplotlib widget
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Welcome to the kymograph analysis pipeline module!

Use this notebook to load kymographs and extract transport flows from them. The first step is to supply some file parameters to make sure that the measurements are connected to real metrics and that the segmentation method is correct.

Important to note: 
Seg-Tresh is the limit of what percentage of the segmented image should be hypha. 

In [33]:
### Please supply a folder with kymographs. You can filter it based on the title. 
### Doing that makes it easier to set image parameters for each kymo group.
### Spatial resolution should be in micrometers

img_address = 'E:\\AMOLF_Data\\single_tiffs\\*.tif'
out_address = Path('E:\\AMOLF_Data\\single_tiffs_output\\')
    
# space_resolution is in micrometers per pixel,
# fps is in frames per second
# space_resolution = 4*1.725 / 50
space_resolution = 0.8625
fps = 10
display_kymos=True

address_list = [Path(address) for address in glob(img_address)]
kymo_stack  = [imageio.imread(address) for address in address_list]
kymo_edge_list = [KymoEdgeAnalysis(kymo=kymo, edge_name=address_list[i].stem, address=out_address) for i, kymo in enumerate(kymo_stack)]

for i, kymo_anal in enumerate(tqdm(kymo_edge_list)):
    title = address_list[i].name.split('.')[-2]
    imshow_extent = [0, space_resolution * kymo_anal.kymo.shape[1],
                1/fps * kymo_anal.kymo.shape[0], 0]
    kymo_anal.space_pixel_size = space_resolution
    kymo_anal.time_pixel_size = 1/fps    
    forw_thresh, back_thresh = kymo_anal.fourier_kymo(1, test_plots=False)
    speeds, times = kymo_anal.extract_speeds(15, w_start=3, C_thresh=0.95, C_thresh_falloff = 0.00, blur_size = 3, preblur=True, speed_thresh=80)
    flux_tot = kymo_anal.extract_transport()

100%|██████████| 9/9 [06:50<00:00, 45.66s/it]


In [35]:
for i, edge in enumerate(kymo_edge_list):
    edge.edge_name = (i,i)
plot_summary(kymo_edge_list)
save_raw_data(kymo_edge_list, out_address)

C:\GitHub_Projects\AMOLF\amftrack\pipeline\development\high_mag_videos\plot_data.py:135: RuntimeWarning: divide by zero encountered in true_divide
  vel_adj = np.where(np.isinf(np.divide(spd_tiff[2], kymo_tiff[1])), np.nan, np.divide(spd_tiff[2], kymo_tiff[1]))
C:\GitHub_Projects\AMOLF\amftrack\pipeline\development\high_mag_videos\plot_data.py:56: RuntimeWarning: divide by zero encountered in true_divide
  vel_adj = np.where(np.isinf(np.divide(spd_tiff[2], kymo_tiff[1])), np.nan, np.divide(spd_tiff[2], kymo_tiff[1]))
C:\GitHub_Projects\AMOLF\amftrack\pipeline\development\high_mag_videos\plot_data.py:56: RuntimeWarning: divide by zero encountered in true_divide
  vel_adj = np.where(np.isinf(np.divide(spd_tiff[2], kymo_tiff[1])), np.nan, np.divide(spd_tiff[2], kymo_tiff[1]))
C:\GitHub_Projects\AMOLF\amftrack\pipeline\development\high_mag_videos\plot_data.py:56: RuntimeWarning: divide by zero encountered in true_divide
  vel_adj = np.where(np.isinf(np.divide(spd_tiff[2], kymo_tiff[1])), n